In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Housing.csv to Housing.csv
Saving train (1).csv to train (1).csv


In [ ]:
import polars as pl
import pandas as pd
import bottleneck as bn

In [ ]:
df_pl = pl.read_csv("train.csv")
df_pl.head()

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,0,3,"""Braund, Mr. Owen Harris""","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
2,1,1,"""Cumings, Mrs. John Bradley (Fl…","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
3,1,3,"""Heikkinen, Miss. Laina""","""female""",26.0,0,0,"""STON/O2. 3101282""",7.925,null,"""S"""
4,1,1,"""Futrelle, Mrs. Jacques Heath (…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S"""
5,0,3,"""Allen, Mr. William Henry""","""male""",35.0,0,0,"""373450""",8.05,null,"""S"""


In [ ]:
df_pl.dtypes
df_pl.null_count()
df_pl.describe()

statistic,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
str,f64,f64,f64,str,str,f64,f64,f64,str,f64,str,str
"""count""",891.0,891.0,891.0,"""891""","""891""",714.0,891.0,891.0,"""891""",891.0,"""204""","""889"""
"""null_count""",0.0,0.0,0.0,"""0""","""0""",177.0,0.0,0.0,"""0""",0.0,"""687""","""2"""
"""mean""",446.0,0.383838,2.308642,null,null,29.699118,0.523008,0.381594,null,32.204208,null,null
"""std""",257.353842,0.486592,0.836071,null,null,14.526497,1.102743,0.806057,null,49.693429,null,null
"""min""",1.0,0.0,1.0,"""Abbing, Mr. Anthony""","""female""",0.42,0.0,0.0,"""110152""",0.0,"""A10""","""C"""
"""25%""",224.0,0.0,2.0,null,null,20.0,0.0,0.0,null,7.925,null,null
"""50%""",446.0,0.0,3.0,null,null,28.0,0.0,0.0,null,14.4542,null,null
"""75%""",669.0,1.0,3.0,null,null,38.0,1.0,0.0,null,31.0,null,null
"""max""",891.0,1.0,3.0,"""van Melkebeke, Mr. Philemon""","""male""",80.0,8.0,6.0,"""WE/P 5735""",512.3292,"""T""","""S"""


In [ ]:
pclass_counts = df_pl.get_column("Pclass").value_counts()
pclass_counts

Pclass,count
i64,u32
1,216
2,184
3,491


In [ ]:
survivors = df_pl.group_by("Sex").agg(pl.col("Survived").sum())
survivors

Sex,Survived
str,i64
"""female""",233
"""male""",109


In [ ]:
older_passengers = df_pl.filter(pl.col("Age") > 44)
older_passengers

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
7,0,1,"""McCarthy, Mr. Timothy J""","""male""",54.0,0,0,"""17463""",51.8625,"""E46""","""S"""
12,1,1,"""Bonnell, Miss. Elizabeth""","""female""",58.0,0,0,"""113783""",26.55,"""C103""","""S"""
16,1,2,"""Hewlett, Mrs. (Mary D Kingcome…","""female""",55.0,0,0,"""248706""",16.0,null,"""S"""
34,0,2,"""Wheadon, Mr. Edward H""","""male""",66.0,0,0,"""C.A. 24579""",10.5,null,"""S"""
53,1,1,"""Harper, Mrs. Henry Sleeper (My…","""female""",49.0,1,0,"""PC 17572""",76.7292,"""D33""","""C"""
…,…,…,…,…,…,…,…,…,…,…,…
858,1,1,"""Daly, Mr. Peter Denis ""","""male""",51.0,0,0,"""113055""",26.55,"""E17""","""S"""
863,1,1,"""Swift, Mrs. Frederick Joel (Ma…","""female""",48.0,0,0,"""17466""",25.9292,"""D17""","""S"""
872,1,1,"""Beckwith, Mrs. Richard Leonard…","""female""",47.0,1,1,"""11751""",52.5542,"""D35""","""S"""


In [ ]:
df_pd = pd.read_csv("train.csv")

In [ ]:
mean_age = bn.nanmean(df_pd["Age"].values)
std_age = bn.nanstd(df_pd["Age"].values)
mean_age, std_age

(29.69911764705882, 14.516321150817317)

In [ ]:
df_pd["Fare_new"] = df_pd["Fare"].apply(lambda x: x * 1.3)

In [ ]:
df_housing = pd.read_csv("Housing.csv")
df_housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [ ]:
initial_memory = df_housing.memory_usage(deep=True).sum()
print(f"Память до оптимизации: {initial_memory / (1024**2):.2f} MB")

Память до оптимизации: 0.25 MB


Price, area, bedrooms, bathrooms и другие числовые столбцы можно преобразовать в более компактные типы:
- price: можно уменьшить до int32 или int16;
- area: можно преобразовать в float32, так как точность float64 не требуется;
- bedrooms, bathrooms, stories, parking: можно уменьшить до int8 или int16, так как значения имеют небольшой диапазон.

In [ ]:
df_housing["price"] = pd.to_numeric(df_housing["price"], downcast="integer")
df_housing["area"] = pd.to_numeric(df_housing["area"], downcast="float")
df_housing["bedrooms"] = pd.to_numeric(df_housing["bedrooms"], downcast="integer")
df_housing["bathrooms"] = pd.to_numeric(df_housing["bathrooms"], downcast="integer")
df_housing["stories"] = pd.to_numeric(df_housing["stories"], downcast="integer")
df_housing["parking"] = pd.to_numeric(df_housing["parking"], downcast="integer")

binary_columns = ["mainroad", "guestroom", "basement", "hotwaterheating", "airconditioning", "prefarea"]
df_housing[binary_columns] = df_housing[binary_columns].astype("bool")

df_housing["furnishingstatus"] = df_housing["furnishingstatus"].astype("category")

In [ ]:
optimized_memory = df_housing.memory_usage(deep=True).sum()
optimized_dtypes = df_housing.dtypes

In [ ]:
print(f"Память после оптимизации: {optimized_memory / (1024**2):.2f} MB")

Память после оптимизации: 0.23 MB


In [ ]:
memory_reduction = (initial_memory - optimized_memory) / initial_memory * 100
print(f"Снижение потребления памяти: {memory_reduction:.2f}%")

Снижение потребления памяти: 7.61%


Выводы:
- После оптимизации потребление памяти уменьшилось на 7.61% (примерно 200 МБ), что означает, что изменение типов столбцов делает работу с данными более эффективной.